In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pickle as pk
import matplotlib as mpl
import json
import pandas as pd
import seaborn as sns

In [ ]:
#city = 'Athens'
#city = 'Barcelona'
#city = 'Budapest'
#city = 'Edinburgh'
#city = 'Glasgow'
#city = 'London'
#city = 'Madrid'
#city = 'Melbourne'
#city = 'NewDelhi'
#city = 'Osaka'
city = 'Perth'
#city = 'Toronto'
#city = 'Vienna'

## Convergence

In [ ]:
file = open('results/' + city + '/reward', 'rb')
reward = pk.load(file)

In [ ]:
len(reward)

In [ ]:
def plot_convergence(reward, city):
    mpl.rcParams['figure.dpi']= 300
    plt.rcParams["figure.figsize"] = [5, 3]
    fig, ax = plt.subplots()
    xpoints = np.array([i for i in range(1, len(reward)+1)])
    ypoints = np.array(reward)

    plt.title('Convergence - ' + city, fontsize=16)
    plt.xlabel("Training episode", fontsize=18)
    plt.ylabel("Average reward", fontsize=18)
    
    plt.grid(linestyle = '-.', axis = 'y')
    
    plt.ylim(0,52)
    plt.xlim(0, 104)
    
    plt.yticks([0, 10, 20, 30, 40, 50], fontsize=16)
    plt.xticks([0, 20, 40, 60, 80, 100], fontsize=16)
    plt.plot(xpoints, ypoints)
    plt.savefig("results/" + city + "/plots/convergence.pdf", bbox_inches='tight')
    plt.savefig("results/" + city + "/plots/convergence.png", bbox_inches='tight')
    plt.show()

In [ ]:
plot_convergence(reward, city)

## Recall and Precision

In [ ]:
f = open('results/' + city + '/75_result.json')
data = json.load(f)

In [ ]:
number_of_users = len(data.keys())

In [ ]:
def intersection(list_a, list_b):
    return [ e for e in list_a if e in list_b ]

In [ ]:
Total_R = []
Total_P = []
Mean_R = 0
Mean_P = 0
instances = 0
for item in data.keys():
    if(len(data[item]['original']) >= 2):
        recall = len(intersection(data[item]['original'], data[item]['recommendation']))/len(data[item]['original'])
        precis = len(intersection(data[item]['original'], data[item]['recommendation']))/len(data[item]['recommendation'])
        Mean_P = Mean_P + precis
        Mean_R = Mean_R + recall
        Total_R.append(recall)
        Total_P.append(precis)
        instances = instances + 1
    
Mean_P = Mean_P / instances
Mean_R = Mean_R / instances

In [ ]:
def plot_cdf_recall_precision(Total_R, Total_P, Mean_R, Mean_P, city):
    #sns.set_theme(style="darkgrid")
    mpl.rcParams['figure.dpi']= 300

    plt.rcParams["figure.figsize"] = [5, 3.5]
    plt.rcParams["figure.autolayout"] = True

    data1 = Total_R
    data2 = Total_P
    
    count1, bins_count1 = np.histogram(data1, bins=100)
    pdf1 = count1 / sum(count1)
    cdf1 = np.cumsum(pdf1)
    plt.plot(bins_count1[1:], cdf1, label="CDF Recall", color='#cc0066')
    plt.plot(0.5, Mean_R, 'o', label="Recall Mean", color='#009933')
    
    count2, bins_count2 = np.histogram(data2, bins=100)
    pdf2 = count2 / sum(count2)
    cdf2 = np.cumsum(pdf2)
    plt.plot(bins_count2[1:], cdf1, label="CDF Precision", color='#111111')
    plt.plot(0.5, Mean_P, 's', label="Precision Mean", color='#009933')
    
    plt.xlabel("Metrics", fontsize=18)
    plt.ylabel("CDF", fontsize=18)
    
    plt.title(city, fontsize=16)
    
    plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1], fontsize=16)
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1], fontsize=16)
    
    plt.grid(linestyle = '--')
    plt.legend(fontsize=12)
    plt.savefig("results/" + city + "/plots/recall_precision.pdf", bbox_inches='tight')
    plt.savefig("results/" + city + "/plots/recall_precision.png", bbox_inches='tight')
    plt.show()

In [ ]:
plot_cdf_recall_precision(Total_R, Total_P, Mean_R, Mean_P, city)

## POI Resurce Usage

In [ ]:
pois = pd.read_csv("data_process/_RawData/" + city + "/POIs.csv")
quantity_of_POIS = len(pois)

POI_to_ID = {}
ID_to_POI = {}
index = 0
for poi in list(pois['poiID']):
    POI_to_ID[poi] = index
    ID_to_POI[index] = poi
    index = index + 1

In [ ]:
def process_data_for_max_mec_band_usage_per_poi(data):
    bandwidth_temporal_usage = np.zeros(shape=(quantity_of_POIS,quantity_of_POIS))
    mec_temporal_usage = np.zeros(shape=(quantity_of_POIS,quantity_of_POIS))
    bandwidth_temporal_usage[:] = np.nan
    mec_temporal_usage[:] = np.nan

    for item in data:
        #print(item)
        t = 0
        for poi in data[item]['recommendation']:
            poi_index = POI_to_ID[poi]
            if(np.isnan(bandwidth_temporal_usage[poi_index][t])):
                bandwidth_temporal_usage[poi_index][t] = 0

            if(np.isnan(mec_temporal_usage[poi_index][t])):
                mec_temporal_usage[poi_index][t] = 0

            bandwidth_temporal_usage[poi_index][t] = bandwidth_temporal_usage[poi_index][t] + data[item]['allocated_bandwidth'][t]
            mec_temporal_usage[poi_index][t] = mec_temporal_usage[poi_index][t] + data[item]['allocated_mec'][t]
            t = t + 1

    max_mec_usage_per_poi = []
    max_bandwidth_usage_per_poi = []
    for item in range(0, quantity_of_POIS):
        max_bandwidth_usage_per_poi.append(np.nanmax(bandwidth_temporal_usage[item]))
        max_mec_usage_per_poi.append(np.nanmax(mec_temporal_usage[item]))
        
    return (max_mec_usage_per_poi, max_bandwidth_usage_per_poi)

In [ ]:
def plot_max_mec_band_usage_per_poi(pois, max_mec_usage, max_band_usage, MEC_CAPACITY, POI_BANDWIDTH_CAPACITY, city):

    fig = plt.figure(figsize = (12,3.5))

    # creating the bar plot
    #plt.bar(pois, max_usage, color ='#006666',
    #        width = 0.8)
    
    #plt.fill_between(pois, max_band_usage, color ='#006699', label='Bandwidth usage')
    #plt.fill_between(pois, max_mec_usage, color ='#ff9900', label='MEC usage')
    
    plt.fill_between(pois, max_band_usage, color ='#006699', label='Bandwidth usage (Mbps)')
    plt.fill_between(pois, max_mec_usage, color ='#cc0066', label='MEC usage (RCs)')
    
    plt.plot(pois, max_mec_usage, color ='#FFFFFF', linewidth=2)
    
    plt.plot([1, len(pois)], [MEC_CAPACITY, MEC_CAPACITY], color='r', linestyle='--', linewidth=4)
    plt.plot([1, len(pois)], [POI_BANDWIDTH_CAPACITY, POI_BANDWIDTH_CAPACITY], color='r', linestyle='--', linewidth=4)
    
    plt.xticks([1, len(pois)/2, len(pois)], [1, "POI ID (" + city + ")", len(pois)], fontsize=28)
    #plt.set_xticklabels(, fontsize=20)
    #plt.yticks([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20])
    ticks = list(range(0, POI_BANDWIDTH_CAPACITY+1, int(POI_BANDWIDTH_CAPACITY/5)))
    plt.yticks(ticks, fontsize=28)
    
    #plt.xlabel("POI (" + city + ")", fontsize=24)
    plt.ylabel("Total demand", fontsize=28)
    #plt.title("Students enrolled in different courses")
    plt.legend(fontsize=28)
    plt.grid(axis = 'y')
    plt.savefig("results/" + city + "/plots/" + str(POI_BANDWIDTH_CAPACITY) + "_mec_band_usage.pdf", bbox_inches='tight')
    plt.savefig("results/" + city + "/plots/" + str(POI_BANDWIDTH_CAPACITY) + "_mec_band_usage.png", bbox_inches='tight')
    plt.show()

In [ ]:
f = open('results/' + city + '/75_result.json')
data = json.load(f)
max_mec_usage_per_poi, max_bandwidth_usage_per_poi = process_data_for_max_mec_band_usage_per_poi(data)
plot_max_mec_band_usage_per_poi(list(pois['poiID']), max_mec_usage_per_poi, max_bandwidth_usage_per_poi, 37.5, 75, city)

In [ ]:
f = open('results/' + city + '/150_result.json')
data = json.load(f)
max_mec_usage_per_poi, max_bandwidth_usage_per_poi = process_data_for_max_mec_band_usage_per_poi(data)
plot_max_mec_band_usage_per_poi(list(pois['poiID']), max_mec_usage_per_poi, max_bandwidth_usage_per_poi, 37.5, 150, city)

In [ ]:
f = open('results/' + city + '/300_result.json')
data = json.load(f)
max_mec_usage_per_poi, max_bandwidth_usage_per_poi = process_data_for_max_mec_band_usage_per_poi(data)
plot_max_mec_band_usage_per_poi(list(pois['poiID']), max_mec_usage_per_poi, max_bandwidth_usage_per_poi, 37.5, 300, city)

## User resource allocation

In [ ]:
def process_data_for_plot_diff(data):
    diff_of_b_allocation = []
    diff_of_m_allocation = []

    for item in data:
        if(data[item]['demand_bandwidth'][1] != 0):
            min_all = round(min(data[item]['allocated_bandwidth']), 6)
            min_dem = data[item]['demand_bandwidth'][0]
            diff_of_b_allocation.append(min_all - min_dem)

        if(data[item]['demand_mec'][1] != 0):
            min_all = round(min(data[item]['allocated_mec']), 6)
            min_dem = data[item]['demand_mec'][0]
            diff_of_m_allocation.append(min_all - min_dem)

    diff_of_b_allocation = np.array(diff_of_b_allocation)
    diff_of_m_allocation = np.array(diff_of_m_allocation)


    non_negative_b_diff = diff_of_b_allocation.copy()
    negative_b_diff = diff_of_b_allocation.copy()

    non_negative_b_diff[non_negative_b_diff < 0] = np.nan
    negative_b_diff[negative_b_diff > 0] = np.nan

    non_negative_m_diff = diff_of_m_allocation.copy()
    negative_m_diff = diff_of_m_allocation.copy()

    non_negative_m_diff[non_negative_m_diff < 0] = np.nan
    negative_m_diff[negative_m_diff >= 0] = np.nan
    
    return (non_negative_b_diff, negative_b_diff, non_negative_m_diff, negative_m_diff)

In [ ]:
def plot_diff_band(non_negative_dif, negative_dif, limit_b, scenario):
    mpl.rcParams['figure.dpi']= 300
    plt.rcParams["figure.figsize"] = [5, 3]
    fig, ax = plt.subplots()
    xpoints = np.array([i for i in range(1, len(non_negative_dif)+1)])
    
    ax.set_title(city + ' - ' + scenario + ' overload',  fontsize=18)
    #plt.xlabel("User ID", fontsize=20)
    plt.ylabel(r'$(b^{t,v}_u - \lambda_{u}^{min})$', fontsize=20)
    
    plt.xticks([1, len(non_negative_dif)/2, len(non_negative_dif)], [1, "User ID", len(non_negative_dif)], fontsize=18)
    plt.yticks(fontsize=18)
    
    #plt.xlabel.label_coords(1.05, -0.025)
    
    pos_dif = str(round((np.count_nonzero(~np.isnan(non_negative_dif))/len(non_negative_dif))*100, 0))
    neg_dif = str(round((np.count_nonzero(~np.isnan(negative_dif))/len(non_negative_dif))*100, 0))
    
    plt.plot(xpoints, non_negative_dif, '.', markerfacecolor='none', markeredgewidth=0.5, label='Positive difference ~ ' + pos_dif + '%')
    plt.plot(xpoints, negative_dif, '.', markerfacecolor='none', markeredgewidth=0.5, label='Negative difference ~ ' +  neg_dif + '%')
    
    plt.legend(fontsize=15, handletextpad=-0.5)
    
    plt.grid(axis = 'y', linestyle='--', linewidth=0.6)
    plt.savefig("results/" + city + "/plots/" + str(limit_b) + "_difference_bandwidth.pdf", bbox_inches='tight')
    plt.savefig("results/" + city + "/plots/" + str(limit_b) + "_difference_bandwidth.png", bbox_inches='tight')
    plt.show()

In [ ]:
def plot_diff_mec(non_negative_dif, negative_dif):
    mpl.rcParams['figure.dpi']= 300
    plt.rcParams["figure.figsize"] = [5, 3]
    fig, ax = plt.subplots()
    xpoints = np.array([i for i in range(1, len(non_negative_dif)+1)])
    
    ax.set_title(city + ' - MEC allocation', fontsize=18)
    #plt.xlabel("User ID", fontsize=20)
    plt.ylabel(r'$(m^{t,v}_u - \psi_{u}^{min})$', fontsize=20)
    plt.xticks([1, len(non_negative_dif)/2, len(non_negative_dif)], [1, "User ID", len(non_negative_dif)], fontsize=18)
    plt.yticks(fontsize=16)
    
    pos_dif = str(round((np.count_nonzero(~np.isnan(non_negative_dif))/len(non_negative_dif))*100, 0))
    neg_dif = str(round((np.count_nonzero(~np.isnan(negative_dif))/len(non_negative_dif))*100, 0))
    
    plt.plot(xpoints, non_negative_dif, '.', markerfacecolor='none', markeredgewidth=0.5, label='Positive difference ~ ' + pos_dif + '%')
    plt.plot(xpoints, negative_dif, '.', markerfacecolor='none', markeredgewidth=0.5, label='Negative difference ~ ' +  neg_dif + '%')
    
    plt.legend(fontsize=15, handletextpad=-0.5)
    
    plt.grid(axis = 'y', linestyle='--', linewidth=0.6)
    plt.savefig("results/" + city + "/plots/difference_mec.pdf", bbox_inches='tight')
    plt.savefig("results/" + city + "/plots/difference_mec.png", bbox_inches='tight')
    plt.show()

In [ ]:
f = open('results/' + city + '/75_result.json')
data = json.load(f)
non_negative_b_diff, negative_b_diff, non_negative_m_diff, negative_m_diff = process_data_for_plot_diff(data)
plot_diff_mec(non_negative_m_diff, negative_m_diff)

In [ ]:
f = open('results/' + city + '/75_result.json')
data = json.load(f)
non_negative_b_diff, negative_b_diff, non_negative_m_diff, negative_m_diff = process_data_for_plot_diff(data)
plot_diff_band(non_negative_b_diff, negative_b_diff, 75, 'High')

In [ ]:
f = open('results/' + city + '/150_result.json')
data = json.load(f)
non_negative_b_diff, negative_b_diff, non_negative_m_diff, negative_m_diff = process_data_for_plot_diff(data)
plot_diff_band(non_negative_b_diff, negative_b_diff, 150, 'Medium')

In [ ]:
f = open('results/' + city + '/300_result.json')
data = json.load(f)
non_negative_b_diff, negative_b_diff, non_negative_m_diff, negative_m_diff = process_data_for_plot_diff(data)
plot_diff_band(non_negative_b_diff, negative_b_diff, 300, 'Low')

## Solution Time

In [ ]:
cities = ['Athens', 'Barcelona', 'Budapest', 'Edinburgh', 'Glasgow', 
          'London', 'Madrid', 'Melbourne', 'NewDelhi', 'Osaka', 'Perth', 'Toronto', 'Vienna']
#cities = ['Athens']

In [ ]:
TOTAL_PROCE_TIME = []
TOTAL_CONVO_TIME = []
TOTAL_TRAIN_TIME = []
TOTAL_RECOM_TIME = []
TOTAL_ALLOC_TIME = []

In [ ]:
def read_time_values(city):
    file = open("results/" + city + "/time", "r")
    data = file.readlines()

    process_t = float(data[0].split('=')[1])
    convolution_t = float(data[1].split('=')[1])
    train_t = float(data[2].split('=')[1])
    recommend_t = float(data[3].split('=')[1])
    allocate_t = float(data[4].split('=')[1]) + float(data[5].split('=')[1]) + float(data[6].split('=')[1])
    
    return process_t, convolution_t, train_t, recommend_t, allocate_t

In [ ]:
for item in cities:
    process_t, convolution_t, train_t, recommend_t, allocate_t = read_time_values(item)

    TOTAL_PROCE_TIME.append(process_t)
    TOTAL_CONVO_TIME.append(convolution_t)
    TOTAL_TRAIN_TIME.append(train_t)
    TOTAL_RECOM_TIME.append(recommend_t)
    TOTAL_ALLOC_TIME.append(allocate_t)
    
TOTAL_PROCE_TIME = np.array(TOTAL_PROCE_TIME)
TOTAL_CONVO_TIME = np.array(TOTAL_CONVO_TIME)
TOTAL_TRAIN_TIME = np.array(TOTAL_TRAIN_TIME)
TOTAL_RECOM_TIME = np.array(TOTAL_RECOM_TIME)
TOTAL_ALLOC_TIME = np.array(TOTAL_ALLOC_TIME)

In [ ]:
def plot_time(PROCE_TIME, CONVO_TIME, TRAIN_TIME, RECOM_TIME, ALLOC_TIME, cities):

    Total_p_time = PROCE_TIME + CONVO_TIME + TRAIN_TIME + RECOM_TIME  + ALLOC_TIME
    Total_c_time = CONVO_TIME + TRAIN_TIME + RECOM_TIME  + ALLOC_TIME
    Total_t_time = TRAIN_TIME + RECOM_TIME  + ALLOC_TIME
    Total_r_time = RECOM_TIME  + ALLOC_TIME
    Total_a_time = ALLOC_TIME
    
    fig = plt.figure(figsize = (10,3))

    plt.bar(cities, Total_p_time, color ='blue', label='Processing time')
    plt.bar(cities, Total_c_time, color ='#009999', label='Convolution time')
    plt.bar(cities, Total_t_time, color ='crimson', label='RL training time')
    plt.bar(cities, Total_r_time, color ='#ff9933', label='Recommendation time')
    plt.bar(cities, Total_a_time, color ='#009900', label='Allocation time')
    
    plt.ylabel("Time (s)", fontsize=20)
    
    plt.xticks(rotation=26, fontsize=17)
    plt.yticks(fontsize=18)
    plt.yscale('log')

    plt.legend(fontsize=12)
    plt.grid(axis = 'y')
    plt.savefig("results/time.pdf", bbox_inches='tight')
    plt.savefig("results/time.png", bbox_inches='tight')
    plt.show()

In [ ]:
plot_time(TOTAL_PROCE_TIME, TOTAL_CONVO_TIME, TOTAL_TRAIN_TIME, TOTAL_RECOM_TIME, TOTAL_ALLOC_TIME, cities)